In [9]:
import pandas as pd
import numpy as np

In [43]:
def data_cleaning(paths, name):
    ### COLLECT DATA RAW
    dataraw = []
    for path in paths:
        with open(path) as f:
            lines = f.readlines()
        dataraw = dataraw + lines
        f.close()
        
    print("len of datas:",len(dataraw))
        
    ### CONVERT To DataFrame
    all_feats = []
    all_dicts = []
    for l in dataraw:
        dict_ = eval(l[1:-2])
        try:
            dict_['Price per m²'] = dict_.pop('Price per mÂ²')
        except:
            pass
        all_dicts.append(dict_)
        for k in dict_.keys():
            # if 'Price per' in k:
            #     k='Price per m²'
            if k not in all_feats:
                all_feats.append(k)
    
    
    data = pd.DataFrame(all_dicts,columns=all_feats)
    
    ### DATA Cleaning
    for col in list(data.columns):
        if "price" in col or "Price" in col or "Rates and Taxes" in col or "Levies" in col:
            if col == "price":
                print("Drop POA values")
                data = data[data[col]!="POA"]
            data[col] = data[col].str.lstrip("R ")
            data[col] = data[col].replace(r" ","",regex=True)
            data[col] = pd.to_numeric(data[col])
        
        elif "Size" in col or "size" in col :
            data[col] = data[col].str.rstrip(" m²")
            data[col] = data[col].str.rstrip(" m�")
            data[col] = data[col].str.rstrip(" m\B2")
            data[col] = data[col].str.rstrip(" mÂ")
            data[col] = data[col].replace(r" ","",regex=True)
            data[col] = data[col].replace(r"ha","",regex=True)
            data[col] = data[col].replace(r"acres","",regex=True)
            data[col] = data[col].replace(r"m²","",regex=True)
            data[col] = pd.to_numeric(data[col])

        elif "Coverage" in col :
            data[col] = data[col].str.rstrip(" %")
            data[col] = data[col].replace(r" ","",regex=True)
            data[col] = pd.to_numeric(data[col])

        elif "Height Restriction" in col:
            data[col] = data[col].str.rstrip(" m²")
            data[col] = data[col].str.rstrip(" m�")
            data[col] = data[col].str.rstrip(" m\B2")
            data[col] = data[col].str.rstrip(" mÂ")
            data[col] = data[col].replace(r" ","",regex=True)
            data[col] = data[col].replace(r"m²","",regex=True)
            data[col] = pd.to_numeric(data[col])

        elif "Date" in col or "date" in col:
            data[col] = pd.to_datetime(data[col])

        ### Format list values
        elif data[col].dtypes == np.object:
            if str(type(data[col][data[col].first_valid_index()])) == "<class 'list'>":
                series=[]
                for value in data[col]:
                    if str(value) != "nan":
                        value = [i.replace('km','') for i in value]
                        value = [float(i) for i in value]
                    series.append(value)
                data[col] = series
    
    #data = data.drop_duplicates()
    
    #data["index"] = data.index
    print("data dimensions:",data.shape)
    save_to = "../../Data/Originals/"+name
    data.to_csv(save_to,sep=";",index=False)

In [44]:
paths = ["../data_collect/data_property_v0.csv","../data_collect/Multi collect/data_property.csv",
         "../data_collect/Multi collect/v2/data_property_v2.csv","../data_collect/Multi collect/v3/data_property_v3.csv",
         "../data_collect/Multi collect/v4/data_property_v4.csv","../data_collect/Multi collect/v5/data_property_v2.csv",
         "../data_collect/Multi collect/v6/data_property_v6.csv","../data_collect/Multi collect/v7/data_property_v2.csv"
        ]

data_cleaning(paths,"Dataset_update.csv")

len of datas: 14111
{'name': '1 Bedroom Apartment / Flat for Sale in Umhlanga Rocks', 'price': 'R 1 630 000', 'province': 'kwazulu-natal', 'city': 'umhlanga', 'address': '38 Umhlanga Ridge Boulevard, Umhlanga Rocks', 'Listing Number': '108813997', 'Type of Property': 'Apartment / Flat', 'Street Address': '38 Umhlanga Ridge Boulevard, Umhlanga Rocks', 'Listing Date': '07 May 2021', 'Floor Size': '68 mÂ²', 'Bedroom': 'Bedroom with sliding doors, built-in cupboards and screeded floors', 'Bathrooms': '1.5', 'Bathroom 1': 'Bathroom 2 with basin, shower and toilet', 'Bathroom 2': 'Bathroom 1 with toilet', 'Kitchen': 'Open plan kitchen with screeded floors, extractor fan and oven and hob', 'Reception Rooms': '1', 'Other': '2', 'Other 1': 'Open plan living room', 'Other 2': 'Laundry', 'Parking': '1', 'Garden': 'Garden', 'Pool': 'Swimming pool', 'Security': '24 hour manned security', 'Special Featuress': '3', 'Special Features 1': 'Complex with gym', 'Special Features 2': 'Walls with timber cla

KeyError: 'Price per mÂ²'